In [1]:
import cv2
import json
import matplotlib.pyplot as plt
import pandas as pd
import random
import shutil


from glob import glob
from tqdm import tqdm

**Mounted Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/miniProject/benetech-making-graphs-accessible.zip -d dataset

Streaming output truncated to the last 5000 lines.
  inflating: dataset/train/images/eaed09039206.jpg  
  inflating: dataset/train/images/eaed1e245167.jpg  
  inflating: dataset/train/images/eaedb251e670.jpg  
  inflating: dataset/train/images/eaeead418936.jpg  
  inflating: dataset/train/images/eaeeb1c2ecdd.jpg  
  inflating: dataset/train/images/eaef70ba842d.jpg  
  inflating: dataset/train/images/eaf036991cb3.jpg  
  inflating: dataset/train/images/eaf06e9e07db.jpg  
  inflating: dataset/train/images/eaf20f7d8945.jpg  
  inflating: dataset/train/images/eaf3a9a87868.jpg  
  inflating: dataset/train/images/eaf3d05d20a6.jpg  
  inflating: dataset/train/images/eaf3d24fb032.jpg  
  inflating: dataset/train/images/eaf3f734b5ce.jpg  
  inflating: dataset/train/images/eaf467f871a2.jpg  
  inflating: dataset/train/images/eaf65b4e3c45.jpg  
  inflating: dataset/train/images/eaf7f876411a.jpg  
  inflating: dataset/train/images/eafa35356cf8.jpg  
  inflating: dataset/train/images/eafa65caf8e7.j

**Install PaddleOCR**

In [4]:
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 46466, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 46466 (delta 184), reused 245 (delta 144), pack-reused 46134
Receiving objects: 100% (46466/46466), 342.85 MiB | 17.26 MiB/s, done.
Resolving deltas: 100% (32632/32632), done.
Updating files: 100% (1620/1620), done.


In [5]:
!python3 -m pip install paddlepaddle-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.9/584.9 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 43.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.11.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery 3.9.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, b

**Prepare dataset**

In [6]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [7]:
def extract_label_bbox(data, rows, img_id,image = "None"):
    common = []
    if data["chart-type"] != 'scatter':
      return None
    common+= [img_id, data["source"], data["chart-type"]]
    # TRAIN_FOLDER = "/content/dataset/train/images/"
    # image = cv2.imread(TRAIN_FOLDER + img_id + ".jpg")
    image_w,image_h,c = image.shape
    # print(image_w,image_h)
    for box in data["text"]:
        # x0 = box["polygon"]["x0"]
        # y0 = box["polygon"]["y1"]
        # w = abs(box["polygon"]["x1"] - box["polygon"]["x0"]) + abs(box["polygon"]["x2"] - box["polygon"]["x1"])
        # h = abs(box["polygon"]["y3"] - box["polygon"]["y1"])
        # x0 = box["polygon"]["x0"]
        # y0 = box["polygon"]["y0"]
        # w = box["polygon"]["x1"] - box["polygon"]["x0"]
        # h = box["polygon"]["y3"] - box["polygon"]["y0"]
        # print(box["polygon"]["x2"],box["polygon"]["x0"],box["polygon"]["y2"],box["polygon"]["y0"])
        label = box["role"]
        if label != "tick_label":
          continue
        x0 = (box["polygon"]["x0"] + box["polygon"]["x2"])/2
        y0 = (box["polygon"]["y0"] + box["polygon"]["y2"])/2
        w = box["polygon"]["x2"] - box["polygon"]["x0"]
        h = box["polygon"]["y3"] - box["polygon"]["y0"]
        x0 = x0/image_h
        y0 = y0/image_w
        w = w/image_h
        h = h/image_w

        text = box["text"]
        # print(label)
        box_row = common + [x0, y0, w, h, label,text]
        rows.append(box_row)
    return rows

In [8]:
ANNOTATION = "/content/dataset/train/annotations/*.json"
TRAIN_FOLDER = "/content/dataset/train/images/"
rows = []
for file_name in tqdm(glob(ANNOTATION)):
    label_bbox = []
    tick_bbox = []
    with open(file_name) as f:
        data = json.load(f)
        img_id = file_name.split("/")[-1].split(".")[0]
        image = cv2.imread(TRAIN_FOLDER + img_id + ".jpg")
        label_bbox = extract_label_bbox(data, label_bbox, img_id,image)
        if label_bbox == None:
           continue
        data_rows = label_bbox
    rows += data_rows
    # break

100%|██████████| 60578/60578 [03:06<00:00, 325.13it/s]


In [9]:
df = pd.DataFrame(rows)
df.columns = ["image_id", "source", "chart", "x0", "y0", "w", "h", "label","text"]
df.head()

,image_id,source,chart,x0,y0,w,h,label,text
0,a78edf66d27c,generated,scatter,0.114087,0.891429,0.053571,0.028571,tick_label,1940
1,a78edf66d27c,generated,scatter,0.250992,0.891429,0.053571,0.028571,tick_label,1960
2,a78edf66d27c,generated,scatter,0.387897,0.891429,0.053571,0.028571,tick_label,1980
3,a78edf66d27c,generated,scatter,0.523810,0.891429,0.059524,0.028571,tick_label,2000
4,a78edf66d27c,generated,scatter,0.661706,0.891429,0.057540,0.028571,tick_label,2020


In [10]:
def get_stoi(df):
    """Get String to Index dictionary"""
    stoi = {}
    for idx, string in enumerate(df.label.unique()):
        stoi[string] = idx
    itos = {item[1]: item[0] for item in stoi.items()}
    df = df.replace({"label": stoi})
    return stoi

stoi = get_stoi(df)
print(stoi)
NUM_CLASSES =  len(stoi)
df = df[df["label"].isin(list(stoi.keys()))]
df = df.replace({"label": stoi})

{'tick_label': 0}


In [11]:
def get_new_annotation(df):
    for tup in df.groupby("image_id"):
        name = tup[0]
        x = []
        y = []
        w = []
        h = []
        label = []
        for ele in tup:
          try:
            x = ele['x0'].to_list()
            y = ele['y0'].to_list()
            w = ele['w'].to_list()
            h = ele['h'].to_list()
            label = ele['label'].to_list()
            text = ele['text'].to_list()
          except:
            continue
        with open('new_annotation/{}.txt'.format(name),'a') as f:
            for i in range(len(x)):
                f.write("{} {} {} {} {}###{}\n".format(label[i],x[i],y[i],w[i],h[i],text[i]))

In [12]:
!rm -r new_annotation
!mkdir new_annotation
get_new_annotation(df)

rm: cannot remove 'new_annotation': No such file or directory


In [13]:
annotations = [os.path.join('new_annotation', x) for x in os.listdir('new_annotation') if x[-3:] == "txt"]

In [14]:
!rm -r dataCRNN
!mkdir dataCRNN

rm: cannot remove 'dataCRNN': No such file or directory


In [15]:
# annotations = '/content/new_annotation/*'
import matplotlib.patches as patches
class_id_to_name_mapping = {v : k for k,v in stoi.items()}
# print(itos)
# fig, ax = plt.subplots()
# plt.rcParams["figure.figsize"] = (20,15)
def plot_bounding_box(image, annotation_list,text_util):
    annotations = np.array(annotation_list)
    w, h = image.size
    # print(image.size)
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    text_annotations = np.copy(text_util)
    # print(transformed_annotations)
    # print(transformed_annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann,text in zip(transformed_annotations,text_util):
        # obj_cls, x0, y0, w, h = int(ann[0]),int(ann[1]),int(ann[2]),int(ann[3]),int(ann[4])
        obj_cls, x0, y0, x1, y1 = ann
        # print(x0,y0,x1,y1)
        image_patch = np.array(image)[int(y0):int(y1),int(x0):int(x1)]
        # print(class_id_to_name_mapping[(int(obj_cls))])
        if class_id_to_name_mapping[(int(obj_cls))] == "axis_title":
          w = x1 - x0
          h = y1 - y0
          if w < h:
            rotated_img = cv2.rotate(image_patch, cv2.ROTATE_90_CLOCKWISE)
            # plt.imshow(rotated_img)
            # plt.show()
            cv2.imwrite(os.path.join('/content/dataCRNN/', f'{text}.jpg'),  rotated_img)
          else:
            # plt.imshow(image_patch)
            # plt.show()
            cv2.imwrite(os.path.join('/content/dataCRNN/', f'{text}.jpg'), image_patch)
        # plotted_image.rectangle(((x0,y0), (x1,y1)),outline = "black")
        else:

            cv2.imwrite(os.path.join('/content/dataCRNN/', f'{text}.jpg'), image_patch)
            # plt.imshow(image_patch)
            # print(text,image_patch.shape)
            # plt.show()
        # plotted_image.point((x0,y0),fill='black')

        # plotted_image.text((x0, y0 + 10), class_id_to_name_mapping[(int(obj_cls))], (128, 255, 128))
        # print( class_id_to_name_mapping[(int(obj_cls))])
    
    # plt.imshow(np.array(image))
    # plt.show()

# Get any random annotation file 
for annotation_file in tqdm(annotations):
    try:
    # annotation_file = random.choice(annotations)
      with open(annotation_file, "r") as file:
          annotation_list = file.read().split("\n")[:-1]
          annotation_list = [x.split("###") for x in annotation_list]
          # print(annotation_list)
          annotation_util = [ele[0] for ele in annotation_list]
          text_util = [ele[1] for ele in annotation_list]
          annotation_util = [[float(y) for y in x.split(' ')] for x in annotation_util]
          # annotation_list = [ele + [ele2] for ele,ele2 in zip(annotation_util,text_util)]
          # print(annotation_list)
          
      # print(annotation_list)
      #Get the corresponding image file
      # print(annotation_file)
      image_file = annotation_file.replace("new_annotation", "dataset/train/images").replace("txt", "jpg")
      assert os.path.exists(image_file)

      #Load the image
      image = Image.open(image_file)

      #Plot the Bounding Box
      plot_bounding_box(image, annotation_util,text_util)
    except Exception as e:
      # print(image_file)
      # print("An error occurred:", e)
      continue

100%|██████████| 11237/11237 [02:38<00:00, 70.77it/s]


**Custom dataset**

In [16]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(712)

In [17]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18

import string
from tqdm.notebook import tqdm
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import multiprocessing as mp
import glob
import os 
import string
from pathlib import Path

from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18
import matplotlib.pyplot as plt
import collections

from IPython.display import clear_output
%matplotlib inline
plt.style.use('seaborn')

<ipython-input-17-81c56c00e579>:41: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [18]:
data_path = "/content/dataCRNN"

In [19]:
image_fns = os.listdir(data_path)
print(len(image_fns))
print(np.unique([len(".".join(image_fn.split(".")[:-1])) for image_fn in image_fns]))

2507
[1 2 3 4 5 6 7 8 9]


In [20]:
image_fns_train, image_fns_val = train_test_split(image_fns, random_state=0,test_size = 0.1)
image_fns_val, image_fns_test = train_test_split(image_fns_val, random_state=0,test_size = 0.5)
print(len(image_fns_train),len(image_fns_val),len(image_fns_test))

2256 125 126


In [21]:
image_ns = [".".join(image_fn.split(".")[:-1]) for image_fn in image_fns]

In [22]:
image_ns = [".".join(image_fn.split(".")[:-1]) for image_fn in image_fns]
image_ns = "".join(image_ns)
letters = sorted(list(set(list(image_ns))))
print(len(letters))
print(letters)

18
[' ', '$', '%', '+', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'E']


In [23]:
vocabulary = ["#"] + letters
print(len(vocabulary))
print(vocabulary)
idx2char = {k:v for k,v in enumerate(vocabulary, start=0)}
print(idx2char)
char2idx = {v:k for k,v in idx2char.items()}
print(char2idx)

19
['#', ' ', '$', '%', '+', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'E']
{0: '#', 1: ' ', 2: '$', 3: '%', 4: '+', 5: ',', 6: '-', 7: '.', 8: '0', 9: '1', 10: '2', 11: '3', 12: '4', 13: '5', 14: '6', 15: '7', 16: '8', 17: '9', 18: 'E'}
{'#': 0, ' ': 1, '$': 2, '%': 3, '+': 4, ',': 5, '-': 6, '.': 7, '0': 8, '1': 9, '2': 10, '3': 11, '4': 12, '5': 13, '6': 14, '7': 15, '8': 16, '9': 17, 'E': 18}


In [24]:
image_fns_train = ['dataCRNN/'+ ele for ele in image_fns_train]
image_fns_val = ['dataCRNN/'+ ele for ele in image_fns_val]
image_fns_test = ['dataCRNN/'+ ele for ele in image_fns_test]

In [25]:
!rm -r img2text
!mkdir img2text
!mkdir img2text/train
!mkdir img2text/val
!mkdir img2text/test

def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

move_files_to_folder(image_fns_train, 'img2text/train')
move_files_to_folder(image_fns_val, 'img2text/val')
move_files_to_folder(image_fns_test, 'img2text/test')

rm: cannot remove 'img2text': No such file or directory


In [31]:
with open('/content/img2text/train/train.txt','a') as f:
    for image in os.listdir('/content/img2text/train'):
        text = ".".join(image.split(".")[:-1])
        gt = f'{image}, "{text}"\n'
        f.write(gt)

In [33]:
with open('/content/img2text/val/val.txt','a') as f:
    for image in os.listdir('/content/img2text/val'):
        text = ".".join(image.split(".")[:-1])
        gt = f'{image}, "{text}"\n'
        f.write(gt)

In [34]:
with open('/content/img2text/test/test.txt','a') as f:
    for image in os.listdir('/content/img2text/test'):
        text = ".".join(image.split(".")[:-1])
        gt = f'{image}, "{text}"\n'
        f.write(gt)

**Config model**

In [37]:
with open('/content/dictionary.txt','a') as f:
    for ele in letters:
      gt = f'{ele}\n'
      f.write(gt)

In [39]:
%cd PaddleOCR/
# Download the pre-trained model of en_PP-OCRv3
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar
# Decompress model parameters
%cd pretrain_models
!tar -xf en_PP-OCRv3_rec_train.tar && rm -rf en_PP-OCRv3_rec_train.tar

/content/PaddleOCR
--2023-06-11 14:47:09--  https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 36.110.192.138, 36.110.192.178, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|36.110.192.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204093440 (195M) [application/x-tar]
Saving to: ‘./pretrain_models/en_PP-OCRv3_rec_train.tar’

en_PP-OCRv3_rec_tra 100%[===================>] 194.64M  31.6MB/s    in 12s     

2023-06-11 14:47:22 (16.3 MB/s) - ‘./pretrain_models/en_PP-OCRv3_rec_train.tar’ saved [204093440/204093440]

/content/PaddleOCR/pretrain_models


In [45]:
!pip install mmcv-full==1.3.13 -f https://download.openmmlab.com/mmcv/dist/cu102/torch1.9.0/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu102/torch1.9.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.0/313.0 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 21.8 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 417, in run
    _, build_failures = build(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 320, in build
    wheel_file = _buil

In [44]:
%cd /content/
!python3 /PaddleOCR/tools/train.py -c /PaddleOCR/configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model=/PaddleOCR/en_PP-OCRv3_rec_train/best_accuracy

#specify the card number through --gpus
!python3 -m paddle.distributed.launch --gpus '0,1,2,3'  /PaddleOCR/tools/train.py -c /PaddleOCR/configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml -o Global.pretrained_model=/PaddleOCR/en_PP-OCRv3_rec_train/best_accuracy

/content
python3: can't open file '/PaddleOCR/tools/train.py': [Errno 2] No such file or directory
Error: Can not import paddle core while this file exists: /usr/local/lib/python3.10/dist-packages/paddle/fluid/libpaddle.so
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/lib/python3.10/dist-packages/paddle/__init__.py", line 25, in <module>
    from .framework import monkey_patch_variable
  File "/usr/local/lib/python3.10/dist-packages/paddle/framework/__init__.py", line 17, in <module>
    from . import random  # noqa: F401
  File "/usr/local/lib/python3.10/dist-packages/paddle/framework/random.py", line 16, in <module>
    import paddle.fluid as fluid
  File "/usr/local/lib/python3.10/dist-packages/paddle/fluid/__init__.py", line 36, in <module>

In [42]:
%pwd

'/content/PaddleOCR'